In [59]:
# pip install selenium 
# or 

# conda install -c conda-forge selenium

# Download a chromedriver
#http://chromedriver.storage.googleapis.com/index.html      
# Recommend: version 2.35/ chromedriver_mac64.zip

#Selenium is a web browser testing automation tool
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

import os
# mv chrome driver from Downloads to Applications 
chromedriver = "/Users/audreybaker/Downloads/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver


driver = webdriver.Chrome(chromedriver)
#driver.get("https://mail.google.com")

# Alternatives to Chrome:
# Firefox, phantomjs

import numpy as np
import pandas as pd
import time
import pickle

In [60]:
#Navigate to original page
fec_url = "https://www.fec.gov/data/candidates/house/"
driver.get(fec_url)

In [61]:
#sleep function, import sleep 'sleep(3)'

#2016
cycle_selector = '//select[@name="cycle"]/option[@value=2016]'
driver.find_element_by_xpath(cycle_selector).click()

#results per page to 100
num_results = '//select[@name="results_length"]/option[@value=100]'
driver.find_element_by_xpath(num_results).click()

rec_dis_2016 = []

time.sleep(3)

In [62]:
#pull down all data
data_selector = '//table//tr'

g=driver.find_elements_by_xpath(data_selector)

rec_dis_2016 = []
for row in g:
    li = []
    for e in row.find_elements_by_tag_name('td'):
        #print(e.text)
        li.append(e.text)
    if len(li)==7:
        rec_dis_2016.append(li)


In [63]:
while True:
  
    try:
        driver.find_element_by_xpath('//a[@class="paginate_button next"]').click()
    except:
        break
        
    time.sleep(3)
    data_selector = '//table//tr'

    g=driver.find_elements_by_xpath(data_selector)

    for row in g:
        li = []
        for e in row.find_elements_by_tag_name('td'):
            #print(e.text)
            li.append(e.text)
        if len(li)==7:
            rec_dis_2016.append(li)
    

In [64]:
print(rec_dis_2016)

#len(rec_dis_2016[1])

[['RYAN, PAUL D.', 'REPUBLICAN PARTY', 'WI', '01', '$19,852,220.75', '$13,433,668.84', ''], ['TRONE, DAVID', 'DEMOCRATIC PARTY', 'MD', '08', '$13,421,964.36', '$13,386,290.19', ''], ['PERKINS, RANDY', 'DEMOCRATIC PARTY', 'FL', '18', '$10,873,242.18', '$10,862,988.41', ''], ['MCSALLY, MARTHA E. MS.', 'REPUBLICAN PARTY', 'AZ', '02', '$7,878,156.18', '$8,004,418.35', ''], ['MCCARTHY, KEVIN', 'REPUBLICAN PARTY', 'CA', '23', '$7,824,640.59', '$8,161,454.49', ''], ['ZINKE, RYAN K', 'REPUBLICAN PARTY', 'MT', '00', '$5,680,723.67', '$6,064,302.48', ''], ['DOLD, ROBERT JAMES JR', 'REPUBLICAN PARTY', 'IL', '10', '$5,488,206.47', '$5,650,246.20', ''], ['LOVE, MIA', 'REPUBLICAN PARTY', 'UT', '04', '$5,472,751.26', '$5,576,207.69', ''], ['COMSTOCK, BARBARA J HONORABLE', 'REPUBLICAN PARTY', 'VA', '10', '$5,331,366.52', '$5,342,922.33', ''], ['SCHNEIDER, BRADLEY SCOTT', 'DEMOCRATIC PARTY', 'IL', '10', '$5,197,782.27', '$5,057,592.74', ''], ['ROONEY, FRANCIS', 'REPUBLICAN PARTY', 'FL', '19', '$5,147,0

In [322]:
fec2016 = pd.DataFrame(rec_dis_2016)

In [323]:
fec2016['year'] = 2016


fec2016.columns = ['candidate', 'party', 'state', 'district',\
                   'receipts', 'disbursements', 'delete', 'year']

fec2016 = fec2016.drop(['delete'], axis=1)


In [324]:
#get dollar amounts into floats
fec2016[['receipts','disbursements']] = fec2016[['receipts','disbursements']].replace('[\$,]','',regex=True).replace('[--]','',regex=True)

fec2016.receipts = pd.to_numeric(fec2016.receipts)
fec2016.disbursements = pd.to_numeric(fec2016.disbursements)
fec2016.district = pd.to_numeric(fec2016.district)



In [325]:
#calculate district expenditures
fec_district = fec2016.groupby(['state','district'],as_index=False)['receipts','disbursements'].sum()
#rename district spending column
fec_district = fec_district.rename(columns = {'receipts': 'district_receipts', 'disbursements':'district_disbursements'})
#merge district votes back onto elections dataframe
fec2016 = pd.merge(fec2016,fec_district, how = 'left', on=['state', 'district'])

In [326]:
fec2016.district.unique()

array([  1.,   8.,  18.,   2.,  23.,   0.,  10.,   4.,  19.,   3.,   6.,
         9.,   5.,  12.,  39.,   7.,  11.,  26.,  17.,  52.,  36.,  15.,
        24.,  21.,  14.,  22.,  31.,  49.,  13.,  32.,  16.,  33.,  45.,
        44.,  34.,  29.,  25.,  28.,  30.,  27.,  38.,  42.,  20.,  50.,
        51.,  nan,  41.,  35.,  43.,  46.,  48.,  47.,  40.,  37.,  53.])

In [327]:
#fec2016.district = pd.to_numeric(fec2016.district)

#fec2016['district'] = fec2016.district.apply(lambda x: x.replace('', '00'))

In [328]:
fec_district = fec2016.groupby(['state', 'district'], as_index = False)['receipts','disbursements'].sum()

In [329]:
fec_district['year']=2016

In [330]:
fec_district['end_dollars'] = fec_district['receipts'] - fec_district['disbursements']

In [331]:
fec_district['year']=2016

In [336]:
fec_district.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 448 entries, 0 to 447
Data columns (total 6 columns):
state            448 non-null object
district         448 non-null float64
receipts         448 non-null float64
disbursements    448 non-null float64
year             448 non-null int64
end_dollars      448 non-null float64
dtypes: float64(4), int64(1), object(1)
memory usage: 24.5+ KB


In [335]:
fec_district
fec_district.to_pickle('fec_2016.pickle')

In [281]:
!ls

2016election.pdf                  csvElection
ACScleaning.ipynb                 district_votes.pickle
ACSyears                          district_votes2014.pickle
ElectionPDFtolist.ipynb           district_votes2016.pickle
Selenium to scrape FEC data.ipynb fec_2016.pickle
Wikidata                          pdfScrape.ipynb
Wikipedia


In [333]:
#fec_district[(fec_district.district == 0.0), 'district'] = 1.0

fec_district.district[fec_district.district ==0.0] = 1.0

/Users/audreybaker/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [117]:
fec2016['candidate_Lname'] = fec2016.candidate.str.split().str.get(0)
fec2016['candidate_Lname'] = fec2016.candidate_Lname.map(lambda x: x.strip(','))



In [118]:
fec2016.head()

,candidate,party,state,district,receipts,disbursements,year,end_dollars,candidate_Lname
0,"RYAN, PAUL D.",REPUBLICAN PARTY,WI,01,19852220.75,13433668.84,2016,6418551.91,RYAN
1,"TRONE, DAVID",DEMOCRATIC PARTY,MD,08,13421964.36,13386290.19,2016,35674.17,TRONE
2,"PERKINS, RANDY",DEMOCRATIC PARTY,FL,18,10873242.18,10862988.41,2016,10253.77,PERKINS
3,"MCSALLY, MARTHA E. MS.",REPUBLICAN PARTY,AZ,02,7878156.18,8004418.35,2016,-126262.17,MCSALLY
4,"MCCARTHY, KEVIN",REPUBLICAN PARTY,CA,23,7824640.59,8161454.49,2016,-336813.90,MCCARTHY


In [126]:
fec2016.sort_values(by = ['state', 'district','year', 'candidate_Lname'])
ids=fec2016.groupby(['state', 'district','year', 'candidate_Lname'], as_index = False).size()

In [131]:
ids.sort_values()

state  district  year  candidate_Lname
AK               2016  LINDBECK           1
NY     13        2016  RANGEL             1
                       POWELL             1
                       PERKINS            1
                       LINARES            1
                       GALLAGHER          1
                       FENSTERMAKER       1
                       ESPAILLAT          1
                       COOK               1
       12        2016  MALONEY            1
                       LINDNER            1
                       EISENBACH          1
       11        2016  REICHARD           1
       13        2016  WILLIAMS           1
       11        2016  RECCHIA            1
                       GRIMM              1
                       GENTILE            1
                       DONOVAN            1
       10        2016  ROSENTHAL          1
                       ROSENBERG          1
                       NADLER             1
                       ITKIS         

In [142]:
mask = ((fec2016["state"] == "CA") & 
(fec2016["district"] == "31") & 
(fec2016["candidate_Lname"] == "BACA"))
fec2016[mask].head()

,candidate,party,state,district,receipts,disbursements,year,end_dollars,candidate_Lname
1098,"BACA, JOE",REPUBLICAN PARTY,CA,31,34598.0,34117.74,2016,480.26,BACA
1791,"BACA, JOE",DEMOCRATIC PARTY,CA,31,0.0,5119.20,2016,-5119.20,BACA


In [286]:
#Navigate to original page
fec_url = "https://www.fec.gov/data/candidates/house/"
driver.get(fec_url)

In [287]:
#sleep function, import sleep 'sleep(3)'

#2014
cycle_selector = '//select[@name="cycle"]/option[@value=2014]'
driver.find_element_by_xpath(cycle_selector).click()

#results per page to 100
num_results = '//select[@name="results_length"]/option[@value=100]'
driver.find_element_by_xpath(num_results).click()

rec_dis_2014 = []

time.sleep(3)

In [288]:
#pull down all data
data_selector = '//table//tr'

g=driver.find_elements_by_xpath(data_selector)

rec_dis_2014 = []
for row in g:
    li = []
    for e in row.find_elements_by_tag_name('td'):
        #print(e.text)
        li.append(e.text)
    if len(li)==7:
        rec_dis_2014.append(li)

In [289]:
while True:
  
    try:
        driver.find_element_by_xpath('//a[@class="paginate_button next"]').click()
    except:
        break
        
    time.sleep(3)
    data_selector = '//table//tr'

    g=driver.find_elements_by_xpath(data_selector)

    for row in g:
        li = []
        for e in row.find_elements_by_tag_name('td'):
            #print(e.text)
            li.append(e.text)
        if len(li)==7:
            rec_dis_2014.append(li)

In [311]:
fec2014 = pd.DataFrame(rec_dis_2014)

In [312]:
fec2014['year'] = 2014


fec2014.columns = ['candidate', 'party', 'state', 'district',\
                   'receipts', 'disbursements', 'delete', 'year']

fec2014 = fec2014.drop(['delete'], axis=1)

In [313]:
#get dollar amounts into floats
fec2014[['receipts','disbursements']] = fec2014[['receipts','disbursements']].replace('[\$,]','',regex=True).replace('[--]','',regex=True)

fec2014.receipts = pd.to_numeric(fec2014.receipts)
fec2014.disbursements = pd.to_numeric(fec2014.disbursements)
fec2014.district = pd.to_numeric(fec2014.district)

In [314]:
fec_district = fec2014.groupby(['state', 'district'], as_index = False)['receipts','disbursements'].sum()

In [315]:
fec_district['year']=2014

In [316]:
fec_district['end_dollars'] = fec_district['receipts'] - fec_district['disbursements']

In [317]:
fec_district = fec_district.copy()
fec_district.district[fec_district.district ==0.0] = 1.0

/Users/audreybaker/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [320]:
fec_district.head()

,state,district,receipts,disbursements,year,end_dollars
0,AK,1.0,1095473.24,1108275.53,2014,-12802.29
1,AL,1.0,5431007.86,5081897.72,2014,349110.14
2,AL,2.0,1106979.56,827038.21,2014,279941.35
3,AL,3.0,1132552.22,966786.50,2014,165765.72
4,AL,4.0,1179691.95,914461.91,2014,265230.04


In [321]:

fec_district.to_pickle('fec_2014.pickle')